In [1]:
import squigglepy as sq
import numpy as np
import pandas as pd
from squigglepy.numbers import K, M, B
from chip_estimates_utils import (
    normalize_shares,
    compute_h100_equivalents,
    print_cumulative_summary,
    estimate_chip_sales,
    summarize_quarterly_by_chip
)

sq.set_seed(42)
np.random.seed(42)
N_SAMPLES = 5000
H100_TOPS = 1979

Spreadsheet with input data for quarterly TPU revenue, based on Broadcom's fiscal calendar, as well as TPU production mix: https://docs.google.com/spreadsheets/d/1CRXA0T7jpyg7tDJNQ4KHIznpkDopPH3krspvXj0OlWk/edit?gid=0#gid=0

In [2]:
# ==============================================
# GOOGLE SHEETS CONFIGURATION
# ==============================================
# Replace SPREADSHEET_ID with your actual Google Sheets ID after uploading
# The ID is the long string in the URL: https://docs.google.com/spreadsheets/d/SPREADSHEET_ID/edit

SPREADSHEET_ID = "1CRXA0T7jpyg7tDJNQ4KHIznpkDopPH3krspvXj0OlWk"

# Sheet GIDs (from the URL's gid= parameter - more reliable than sheet names)
REVENUE_GID = 0  # Default first sheet
PROD_MIX_GID = 913806243

# Construct URLs for direct CSV export using gid
REVENUE_URL = f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/export?format=csv&gid={REVENUE_GID}"
PROD_MIX_URL = f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/export?format=csv&gid={PROD_MIX_GID}"

In [3]:
# ==============================================
# LOAD DATA FROM GOOGLE SHEETS
# ==============================================

revenue_df = pd.read_csv(REVENUE_URL)
print(f"Loaded {len(revenue_df)} quarters of revenue data")
print(revenue_df[['quarter', 'start_date', 'end_date', 'revenue_p5', 'revenue_p95']].head())

prod_mix_df = pd.read_csv(PROD_MIX_URL)
print(f"Loaded {len(prod_mix_df)} version-quarter combinations")
print(prod_mix_df[['quarter', 'version', 'share_p5', 'share_p95']].head(10))

Loaded 12 quarters of revenue data
   quarter  start_date    end_date  revenue_p5  revenue_p95
0  Q1_FY23  10/31/2022   1/29/2023        0.42         0.53
1  Q2_FY23   1/30/2023   4/30/2023        0.53         0.66
2  Q3_FY23    5/1/2023   7/30/2023        0.53         0.66
3  Q4_FY23   7/31/2023  10/29/2023        0.79         0.99
4  Q1_FY24  10/30/2023    2/4/2024        1.25         1.40
Loaded 33 version-quarter combinations
   quarter version  share_p5  share_p95
0  Q1_FY23      v4      0.70       0.85
1  Q1_FY23      v3      0.15       0.30
2  Q2_FY23      v4      0.90       0.99
3  Q2_FY23      v3      0.01       0.10
4  Q3_FY23      v4      0.87       0.98
5  Q3_FY23     v5e      0.01       0.08
6  Q4_FY23      v4      0.80       0.92
7  Q4_FY23     v5e      0.08       0.20
8  Q1_FY24     v5e      0.35       0.60
9  Q1_FY24      v4      0.35       0.60


In [4]:
# ==============================================
# CONVERT TO SQUIGGLEPY DISTRIBUTIONS
# ==============================================

# Convert revenue DataFrame to dictionary of distributions
TPU_REVENUE = {}
for _, row in revenue_df.iterrows():
    quarter = row['quarter']
    TPU_REVENUE[quarter] = sq.norm(row['revenue_p5'], row['revenue_p95'])

print(f"Created {len(TPU_REVENUE)} revenue distributions")
print(f"Quarters: {list(TPU_REVENUE.keys())}")

# Convert production mix DataFrame to nested dictionary of distributions
PROD_MIX = {}
for quarter in prod_mix_df['quarter'].unique():
    quarter_data = prod_mix_df[prod_mix_df['quarter'] == quarter]
    PROD_MIX[quarter] = {}
    for _, row in quarter_data.iterrows():
        version = row['version']
        PROD_MIX[quarter][version] = sq.norm(row['share_p5'], row['share_p95'], lclip = 0, rclip = 1)

print(f"\nCreated production mix for {len(PROD_MIX)} quarters")
print(f"Example Q1_FY23 versions: {list(PROD_MIX['Q1_FY23'].keys())}")

Created 12 revenue distributions
Quarters: ['Q1_FY23', 'Q2_FY23', 'Q3_FY23', 'Q4_FY23', 'Q1_FY24', 'Q2_FY24', 'Q3_FY24', 'Q4_FY24', 'Q1_FY25', 'Q2_FY25', 'Q3_FY25', 'Q4_FY25']

Created production mix for 12 quarters
Example Q1_FY23 versions: ['v4', 'v3']


In [5]:
# ======================
# TPU Specs and Margins
# ======================

# TPU specs: 8-bit TOPS and manufacturing costs
TPU_SPECS = {
    'v3':  {'tops': 123,  'cost': sq.to(940, 1400), 'full_name': 'TPU v3'},
    'v4':  {'tops': 275,  'cost': sq.to(1100, 1500), 'full_name': 'TPU v4'},
    'v5e': {'tops': 393,  'cost': sq.to(950, 1400), 'full_name': 'TPU v5e'},
    'v5p': {'tops': 918,  'cost': sq.to(2300, 2900), 'full_name': 'TPU v5p'},
    'v6e': {'tops': 1836, 'cost': sq.to(1600, 1900), 'full_name': 'TPU v6e'},
    'v7':  {'tops': 4614, 'cost': sq.to(4600, 5500), 'full_name': 'TPU v7'},
}

# Broadcom margins (higher in FY23, lower afterward)
MARGIN_FY23 = sq.to(0.60, 0.75)
MARGIN = sq.to(0.50, 0.70)

In [6]:
def get_price_distribution(version, is_fy23=False):
    """Get price distribution for a TPU version: cost / (1 - margin)."""
    margin = MARGIN_FY23 if is_fy23 else MARGIN
    return TPU_SPECS[version]['cost'] / (1 - margin)

# Pre-compute price distributions for each version and margin regime
PRICE_DIST_FY23 = {version: get_price_distribution(version, is_fy23=True) for version in TPU_SPECS}
PRICE_DIST = {version: get_price_distribution(version, is_fy23=False) for version in TPU_SPECS}

# Define sampling functions which get passed into estimate_chip_sales
def sample_revenue(quarter):
    # draw one sample from the quarter's distribution
    return (TPU_REVENUE[quarter] @ 1) * B

def sample_shares(quarter):
    mix = PROD_MIX[quarter]
    # draw one sample from each version's distribution
    raw_shares = {version: dist @ 1 for version, dist in mix.items()}
    return normalize_shares(raw_shares)

def sample_price(quarter, version):
    price_dists = PRICE_DIST_FY23 if 'FY23' in quarter else PRICE_DIST
    return price_dists[version] @ 1

In [7]:
"""
Run Monte Carlo simulation to estimate chip volumes.

Args:
    quarters: list of quarter identifiers (e.g., ['Q1_FY23', 'Q2_FY23', ...])
    versions: list of chip types (e.g., ['v3', 'v4', 'v5e', ...])
    sample_revenue: fn(quarter) -> float, samples or looks up total chip revenue in dollars for a quarter
    sample_shares: fn(quarter) -> dict, samples {version: share} for a quarter (should sum to 1)
    sample_price: fn(quarter, version) -> float, samples or looks up price for a chip type in a quarter
    n_samples: number of Monte Carlo samples

Returns:
    Dictionary of {quarter: {version: [array of samples of chip unit counts]}}
    To find median, confidence intervals, etc you will need to take the percentiles of the result

Note on cross-quarter correlations:
    The sampling functions are called independently for each quarter within each iteration.
    This means any parameters you want correlated across quarters (e.g., a single margin
    value affecting all quarters) will NOT be correlated by default. To preserve cross-quarter
    correlations, pre-sample those parameters outside this function and have your sampling
    functions reference them.
"""

sim_results = estimate_chip_sales(
    quarters=list(TPU_REVENUE.keys()),
    versions=list(TPU_SPECS.keys()),
    sample_revenue=sample_revenue,
    sample_shares=sample_shares,
    sample_price=sample_price,
    n_samples=N_SAMPLES
)

In [8]:
# Detailed summary with each chip type as separate columns with 90% CI
summary_df = summarize_quarterly_by_chip(sim_results)
print("TPU Production Volumes by Quarter (median with 90% CI)")
print(summary_df.to_string(index=False))

TPU Production Volumes by Quarter (median with 90% CI)
Quarter              v3                 v4                v5e                v5p                v6e              v7            Total
Q1_FY23 30k (19k - 45k)   94k (66k - 126k)                  -                  -                  -               -  124k (95k-158k)
Q2_FY23   9k (2k - 18k) 143k (102k - 188k)                  -                  -                  -               - 152k (111k-198k)
Q3_FY23               - 144k (103k - 192k)      8k (2k - 15k)                  -                  -               - 152k (110k-200k)
Q4_FY23               - 195k (139k - 262k)    34k (19k - 55k)                  -                  -               - 230k (173k-298k)
Q1_FY24               - 198k (131k - 272k) 221k (143k - 312k)     10k (4k - 18k)                  -               - 432k (339k-532k)
Q2_FY24               -                  - 480k (331k - 651k)    63k (40k - 92k)                  -               - 545k (396k-715k)
Q3_FY24       

In [9]:
# Cumulative totals by TPU version
# 
# Note: don't trust the confidence intervals here, because they don't account for correlation across quarters
# This means they are probably too narrow

cumulative = {version: np.zeros(N_SAMPLES) for version in TPU_SPECS}
for quarter in sim_results:
    for version in TPU_SPECS:
        cumulative[version] += np.array(sim_results[quarter][version])

print_cumulative_summary(cumulative, TPU_SPECS, "Cumulative TPU Production (FY23-FY25)")


Cumulative TPU Production (FY23-FY25)
Version           p5          p50          p95
---------------------------------------------
v3           25,891       39,242       56,419
v4          664,302      778,074      897,886
v5e       2,079,883    2,452,963    2,850,134
v5p         694,062      832,639      980,132
v6e       1,050,590    1,286,265    1,543,336
v7           37,337       58,305       85,575
---------------------------------------------
TOTAL     5,063,141    5,458,508    5,863,299


In [10]:
# H100 equivalents (based on 8-bit TOPS)
h100_eq = compute_h100_equivalents(cumulative, TPU_SPECS, H100_TOPS)
print_cumulative_summary(h100_eq, TPU_SPECS, "H100 Equivalents (8-bit TOPS basis)")


H100 Equivalents (8-bit TOPS basis)
Version           p5          p50          p95
---------------------------------------------
v3            1,609        2,439        3,506
v4           92,310      108,120      124,769
v5e         413,033      487,122      565,994
v5p         321,955      386,236      454,654
v6e         974,676    1,193,321    1,431,816
v7           87,051      135,937      199,516
---------------------------------------------
TOTAL     2,106,285    2,318,841    2,544,586


In [11]:
# Export quarterly volumes by version to CSV
# ===========================================

from datetime import datetime

# Get current timestamp for notes
timestamp = datetime.now().strftime("%m-%d-%Y %H:%M")
generated_note = f"Date estimates were generated: {timestamp}"

# map v3, TPUv3, TPU v3 to TPU v3 and so on
def get_tpu_name(version):
    version = version.strip()
    if version.startswith("TPU "):
        return version
    if version.startswith("TPU"):
        return f"TPU {version[3:]}"
    return f"TPU {version}"

# Build a lookup dict from revenue_df for quarter -> (start_date, end_date)
quarter_dates_lookup = {}
for _, row in revenue_df.iterrows():
    q = row['quarter']
    start = row.get('start_date', '')
    end = row.get('end_date', '')
    # Use empty string if values are missing or NaN
    if pd.isna(start):
        start = ''
    if pd.isna(end):
        end = ''
    quarter_dates_lookup[q] = (start, end)

# Create rows for output
rows = []

for quarter in sim_results:
    start_date, end_date = quarter_dates_lookup.get(quarter, ('', ''))
    
    for version in TPU_SPECS:
        arr = np.array(sim_results[quarter][version])
        if arr.sum() > 0:
            # Calculate H100 equivalents
            h100e_arr = arr * (TPU_SPECS[version]['tops'] / H100_TOPS)
            
            rows.append({
                'Name': f"{quarter} - {get_tpu_name(version)}",
                'Chip manufacturer': 'Google',
                'Start date': start_date,
                'End date': end_date,
                'Compute estimate in H100e (median)': int(np.percentile(h100e_arr, 50)),
                'H100e (5th percentile)': int(np.percentile(h100e_arr, 5)),
                'H100e (95th percentile)': int(np.percentile(h100e_arr, 95)),
                'Number of Units': int(np.percentile(arr, 50)),
                'Number of Units (5th percentile)': int(np.percentile(arr, 5)),
                'Number of Units (95th percentile)': int(np.percentile(arr, 95)),
                'Source / Link': '',
                'Notes': generated_note,
                'Chip type': get_tpu_name(version),
                'Last Modified By': '',
                'Last Modified': '',
            })

# Create output dataframe
tpu_timelines = pd.DataFrame(rows)

# Save to CSV
output_path = 'tpu_chip_timelines.csv'
tpu_timelines.to_csv(output_path, index=False)

print(f"Exported to {output_path}")
print(tpu_timelines[['Name', 'Chip manufacturer', 'Start date', 'End date', 
                      'Compute estimate in H100e (median)', 'H100e (5th percentile)', 
                      'H100e (95th percentile)', 'Number of Units', 'Number of Units (5th percentile)',
                      'Number of Units (95th percentile)', 'Chip type']].to_string())

Exported to tpu_chip_timelines.csv
                 Name Chip manufacturer  Start date    End date  Compute estimate in H100e (median)  H100e (5th percentile)  H100e (95th percentile)  Number of Units  Number of Units (5th percentile)  Number of Units (95th percentile) Chip type
0    Q1_FY23 - TPU v3            Google  10/31/2022   1/29/2023                                1862                    1174                     2811            29965                             18891                              45227    TPU v3
1    Q1_FY23 - TPU v4            Google  10/31/2022   1/29/2023                               13015                    9210                    17455            93667                             66282                             125618    TPU v4
2    Q2_FY23 - TPU v3            Google   1/30/2023   4/30/2023                                 554                     101                     1092             8925                              1631                              1